In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ood_metrics

In [ ]:
! pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=bcf1276cd6d6cd5fac4a4b090f5cbda16a9f94b3295c736637d0ef4f48c25c36
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available.")

GPU: Tesla T4


In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/eval'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/eval')

In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/road-anomaly-benchmark-master/methods'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/road-anomaly-benchmark-master/methods')

In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/trained_models'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/trained_models')

In [ ]:
from erfnet import ERFNet
from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/cityscapesScripts-master'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/cityscapesScripts-master')

In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/cityscapesScripts-master/cityscapesscripts/helpers'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/cityscapesScripts-master/cityscapesscripts/helpers')

In [ ]:
import os
import sys
shared_folder_path = '/content/drive/MyDrive/cityscapesScripts-master/cityscapesscripts/preparation'
files_in_shared_folder = os.listdir(shared_folder_path)
sys.path.append('/content/drive/MyDrive/cityscapesScripts-master/cityscapesscripts/preparation')

In [ ]:
from csHelpers import printError

In [ ]:
os.environ['CITYSCAPES_DATASET'] = '/content/drive/MyDrive/CITYSCAPES_DATASET'

In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/CITYSCAPES_DATASET/leftImg8bit_trainvaltest.zip'
extracted_path = os.path.dirname(zip_file_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print(f'File unzipped to: {extracted_path}')

File unzipped to: /content/drive/MyDrive/CITYSCAPES_DATASET


In [ ]:
!pip install pyquaternion

In [ ]:
# Code to calculate IoU (mean and per-class) in a dataset
# Nov 2017
# Eduardo Romera
#######################

import numpy as np
import torch
import torch.nn.functional as F
import os
import importlib
import time

from PIL import Image
from argparse import ArgumentParser

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize
from torchvision.transforms import ToTensor, ToPILImage

from dataset import cityscapes
from erfnet import ERFNet
from transform import Relabel, ToLabel, Colorize
from iouEval import iouEval, getColorEntry

NUM_CHANNELS = 3
NUM_CLASSES = 20

image_transform = ToPILImage()
input_transform_cityscapes = Compose([
    Resize(512, Image.BILINEAR),
    ToTensor(),
])
target_transform_cityscapes = Compose([
    Resize(512, Image.NEAREST),
    ToLabel(),
    Relabel(255, 19),   #ignore label to 19
])

def main(args):

    modelpath = args.loadDir + args.loadModel
    weightspath = args.loadDir + args.loadWeights

    print ("Loading model: " + modelpath)
    print ("Loading weights: " + weightspath)

    model = ERFNet(NUM_CLASSES)

    # model = torch.nn.DataParallel(model)
    if (not args.cpu):
      # model = torch.nn.DataParallel(model)
      model = torch.nn.DataParallel(model).cuda()

      def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

    model = load_my_state_dict(model, torch.load(weightspath, map_location=lambda storage, loc: storage))
    print ("Model and weights LOADED successfully")

    model.eval()

    if(not os.path.exists(args.datadir)):
        print ("Error: datadir could not be loaded")

    loader = DataLoader(cityscapes(args.datadir, input_transform_cityscapes, target_transform_cityscapes, subset=args.subset), num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

    iouEvalVal = iouEval(NUM_CLASSES)

    start = time.time()

    for step, (images, labels, filename, filenameGt) in enumerate(loader):

        if (not args.cpu):
            # images = images
            # labels = labels
            images = images.cuda()
            labels = labels.cuda()

        inputs = Variable(images)
        with torch.no_grad():
            outputs = model(inputs)

        iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, labels)

        filenameSave = filename[0].split("leftImg8bit/")[1]

        print (step, filenameSave)


    iouVal, iou_classes = iouEvalVal.getIoU()

    iou_classes_str = []
    for i in range(iou_classes.size(0)):
        iouStr = getColorEntry(iou_classes[i])+'{:0.2f}'.format(iou_classes[i]*100) + '\033[0m'
        iou_classes_str.append(iouStr)

    print("---------------------------------------")
    print("Took ", time.time()-start, "seconds")
    print("=======================================")
    #print("TOTAL IOU: ", iou * 100, "%")
    print("Per-Class IoU:")
    print(iou_classes_str[0], "Road")
    print(iou_classes_str[1], "sidewalk")
    print(iou_classes_str[2], "building")
    print(iou_classes_str[3], "wall")
    print(iou_classes_str[4], "fence")
    print(iou_classes_str[5], "pole")
    print(iou_classes_str[6], "traffic light")
    print(iou_classes_str[7], "traffic sign")
    print(iou_classes_str[8], "vegetation")
    print(iou_classes_str[9], "terrain")
    print(iou_classes_str[10], "sky")
    print(iou_classes_str[11], "person")
    print(iou_classes_str[12], "rider")
    print(iou_classes_str[13], "car")
    print(iou_classes_str[14], "truck")
    print(iou_classes_str[15], "bus")
    print(iou_classes_str[16], "train")
    print(iou_classes_str[17], "motorcycle")
    print(iou_classes_str[18], "bicycle")
    print("=======================================")
    iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
    print ("MEAN IoU: ", iouStr, "%")

if __name__ == '__main__':
    parser = ArgumentParser()

    parser.add_argument('--state')

    parser.add_argument('--loadDir',default="/content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/trained_models/")
    parser.add_argument('--loadWeights', default="erfnet_pretrained.pth")
    parser.add_argument('--loadModel', default="erfnet.py")
    parser.add_argument('--subset', default="val")  #can be val or train (must have labels)
    # parser.add_argument('--datadir', default="C:\\Users\\39347\\Desktop\\DAAI_Project\\CITYSCAPES_DATASET\\")
    parser.add_argument('--datadir', default="/content/drive/MyDrive/CITYSCAPES_DATASET")
    parser.add_argument('--num-workers', type=int, default=4)
    parser.add_argument('--batch-size', type=int, default=1)
    parser.add_argument('--cpu', action='store_true')

    main(parser.parse_args(args=[]))


Loading model: /content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/trained_models/erfnet.py
Loading weights: /content/drive/MyDrive/AnomalySegmentation_CourseProjectBaseCode-main/trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/content/drive/MyDrive/CITYSCAPES_DATASET/leftImg8bit/val /content/drive/MyDrive/CITYSCAPES_DATASET/gtFine/val
0 val/frankfurt/frankfurt_000000_000294_leftImg8bit.png
1 val/frankfurt/frankfurt_000000_000576_leftImg8bit.png
2 val/frankfurt/frankfurt_000000_001016_leftImg8bit.png
3 val/frankfurt/frankfurt_000000_001236_leftImg8bit.png
4 val/frankfurt/frankfurt_000000_001751_leftImg8bit.png
5 val/frankfurt/frankfurt_000000_002196_leftImg8bit.png
6 val/frankfurt/frankfurt_000000_002963_leftImg8bit.png
7 val/frankfurt/frankfurt_000000_003025_leftImg8bit.png
8 val/frankfurt/frankfurt_000000_003357_leftImg8bit.png
9 val/frankfurt/frankfurt_000000_003920_leftImg8bit.png
10 val/frankfurt/frankfurt_000000_004617_leftImg8bit.png


In [ ]:
!pip install cityscapesscripts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=da325ab518395e449fa979f660e8d037b1ebc47ea300bc90badd44ce91da9c46
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


In [ ]:
import os
os.environ['CITYSCAPES_DATASET'] = '/content/drive/MyDrive/CITYSCAPES_DATASET'

In [ ]:
# #!/usr/bin/python
# #
# # Converts the polygonal annotations of the Cityscapes dataset
# # to images, where pixel values encode ground truth classes.
# #
# # The Cityscapes downloads already include such images
# #   a) *color.png             : the class is encoded by its color
# #   b) *labelIds.png          : the class is encoded by its ID
# #   c) *instanceIds.png       : the class and the instance are encoded by an instance ID
# #
# # With this tool, you can generate option
# #   d) *labelTrainIds.png     : the class is encoded by its training ID
# # This encoding might come handy for training purposes. You can use
# # the file labels.py to define the training IDs that suit your needs.
# # Note however, that once you submit or evaluate results, the regular
# # IDs are needed.
# #
# # Uses the converter tool in 'json2labelImg.py'
# # Uses the mapping defined in 'labels.py'
# #

# # python imports
# from __future__ import print_function, absolute_import, division
# import os, glob, sys

# # cityscapes imports
# from cityscapesscripts.helpers.csHelpers import printError
# from cityscapesscripts.preparation.json2labelImg import json2labelImg

# # The main method
# def main():
#     # Where to look for Cityscapes
#     if 'CITYSCAPES_DATASET' in os.environ:
#         cityscapesPath = os.environ['CITYSCAPES_DATASET']
#     else:
#         cityscapesPath = os.path.join(os.path.dirname(os.path.realpath(__file__)),'..','..')
#     # how to search for all ground truth
#     searchFine   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_polygons.json" )
#     searchCoarse = os.path.join( cityscapesPath , "gtCoarse" , "*" , "*" , "*_gt*_polygons.json" )

#     # search files
#     filesFine = glob.glob( searchFine )
#     filesFine.sort()
#     filesCoarse = glob.glob( searchCoarse )
#     filesCoarse.sort()

#     # concatenate fine and coarse
#     files = filesFine + filesCoarse
#     # files = filesFine # use this line if fine is enough for now.

#     # quit if we did not find anything
#     if not files:
#         printError( "Did not find any files. Please consult the README." )

#     # a bit verbose
#     print("Processing {} annotation files".format(len(files)))

#     # iterate through files
#     progress = 0
#     print("Progress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
#     for f in files:
#         # create the output filename
#         dst = f.replace( "_polygons.json" , "_labelTrainIds.png" )

#         # do the conversion
#         try:
#             json2labelImg( f , dst , "trainIds" )
#         except:
#             print("Failed to convert: {}".format(f))
#             raise

#         # status
#         progress += 1
#         print("\rProgress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
#         sys.stdout.flush()


# # call the main
# if __name__ == "__main__":
#     main()

Processing 5000 annotation files
Progress: 100.0 % 